In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import datetime as dt
import daolib.dao as dao
import daolib.dso as dso
import util.sectool as sectool
import util.operatetool as optool
import matplotlib.pyplot as plt
import pickle
import time
import os
%matplotlib inline
from tqdm import tqdm

简单dnn网络构建测试

数据：['mom','emotion', 'pricevol', 'minute', 'finance', 'valuation', 'alpha191','growth']  八大类因子组合测试（共计158个因子+1个标签值）

## 数据读取

In [ ]:
alpha191=readdir('D:\Data\\factor_alpha\\alpha191' )
finance=readdir('D:\Data\\factor_alpha\\finance' )
growth=readdir('D:\Data\\factor_alpha\\growth' )
minute=readdir('D:\Data\\factor_alpha\\minute' )
pricevol=readdir('D:\Data\\factor_alpha\\pricevol' )
valuation=readdir('D:\Data\\factor_alpha\\valuation' )
emotion=readdir('D:\Data\\factor_alpha\\emotion' )
mom=readdir('D:\Data\\factor_alpha\\mom' )

stock_price_df = dao.get_security_info('stock_price_info')
all_stock_list = stock_price_df.index.get_level_values(0).drop_duplicates().tolist()

#股票上市状态(只选择为0的值，不为0的值为异常值)
price_df=dao.get_security_info('stock_price_info')
trade_status_df=price_df.xs('trade_status',level=1)

start_date = dt.datetime(2007, 1, 1)
end_date = dt.datetime(2018,10,1)

In [ ]:
factor_name_list = ['mom','emotion', 'pricevol', 'minute', 'finance', 'valuation', 'alpha191','growth']  

#周频数据
trade_date_w_series = pd.Series( dso.get_trade_date(start_date=start_date, end_date=end_date, period='w') )
week_list = [date for date in trade_date_w_series if dt.datetime(2009,10,20)<date and date < dt.datetime(2018,11,1) ]
week_data = read_factor_data(factors_name_list=factor_name_list, trade_date_20d_series=trade_date_w_series)
week_data['trade_status'] = trade_status_df

In [ ]:
len(week_data)

## 进行预测

In [ ]:
#按照涨跌幅30%划分测试
factor0_df,factor1_df,test_accuracy,train_accuracy=predict_probo_class(week_data,48,week_list,0.3,-1)

## 因子分析

In [ ]:
factor1_df_obj=factor_analyse("test1",factor1_df)
factor1_pre,factor1_unpre,factor1_pre,factor1_unpre=factor_stock_choose(factor1_df,100)

In [ ]:
show2(factor1_df_obj)

In [ ]:
show01(factor1_pre)

## 函数部分

### 预测

In [ ]:
def predict_probo_class(data,loop,date_list,T,x):
    train,test=data_input_class(data,date_list,T,x)  
    predict_value=[]
    test_accuracy=[]
    train_accuracy=[]
    start_clock  = time.time()
    for m in range(len(date_list)-loop):
        x_train, x_test, y_train, y_test,Xx_test= splitdata(train,test,m+loop-1,m,m+loop-1,m+loop)
        y_train=y_train.reshape([len(y_train),1])
        y_test=y_test.reshape([len(y_test),1])
#         x_train= Imputer().fit_transform(x_train)         #填补缺失值 并且进行归一化
#         ss.fit(x_train)
#         x_train = ss.transform(x_train)
        predict = model_create(x_train,y_train,x_test,y_test)
        predict=pd.DataFrame(predict)
        predict['stock']=Xx_test[:,-2]
        predict['date']=Xx_test[:,-1] 
        predict_value.append(predict)
    
    factor0_df,factor1_df=changeindex1(predict_value,loop,date_list)  #??
    return  factor0_df,factor1_df,test_accuracy,train_accuracy     

### 神经网络模型搭建 

In [ ]:
def model_create(x_train,y_train,x_test,y_test):
    #定义输入和输出
    x = tf.placeholder(tf.float32,shape=(None,2),name="x-input")
    y_ = tf.placeholder(tf.float32,shape=(None,1),name="y-input")

    #  ///////////////////隐藏层//////////////////////
    w1 = tf.Variable(tf.truncated_normal([len(x_train), 30], stddev=0.1,seed=1))
    b1 = tf.Variable(tf.zeros([30]))
    L1 = tf.nn.relu(tf.matmul(x, w1) + b1)

    w2 = tf.Variable(tf.truncated_normal([30, 20], stddev=0.1,seed=1))
    b2 = tf.Variable(tf.zeros([20]))
    L2 = tf.nn.relu(tf.matmul(L1, w2) + b2)

    w3 = tf.Variable(tf.truncated_normal([20, 20], stddev=0.1,seed=1))
    b3 = tf.Variable(tf.zeros([20]))
    L3 = tf.nn.relu(tf.matmul(L2, w3) + b3)
    # # ///////////////////隐藏层//////////////////////

    # 输出层
    w4 = tf.Variable(tf.truncated_normal([20, 1], stddev=0.1,seed=1))
    b4 = tf.Variable(tf.zeros([1]))

    y= tf.nn.sigmoid(tf.matmul(L3, w4)+b4)
    #基于min和max对张量t进行截断操作，为了应对梯度爆发或者梯度消失的情况
    cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y,1e-10,1.0))+(1-y_) * tf.log(tf.clip_by_value(1-y,1e-10,1.0)))
    # 使用Adadelta算法作为优化函数，来保证预测值与实际值之间交叉熵最小
    train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)

    with tf.Session() as sess:
        #初始化变量  tf.initialize_all_variables()
            init = tf.initialize_all_variables()
            sess.run(init)
            #设置神经网络的迭代次数
            steps = 500
            for i in range(steps):
            #通过选取样本训练神经网络并更新参数
               for (input_x,input_y) in zip(x_train,y_train):
                   input_x = np.reshape(input_x,(1,2))
                   input_y = np.reshape(input_y,(1,1))
                   sess.run(train_step,feed_dict={x:input_x,y_:input_y})
            #每迭代1000次输出一次日志信息
               if i % 100 == 0:
                # 计算所有数据的交叉熵
                total_cross_entropy = sess.run(cross_entropy,feed_dict={x:x_train,y_:y_train})
                # 输出交叉熵之和
                print("After %d training step(s),cross entropy on all data is %g"%(i,total_cross_entropy))
        #预测输入X的类别
        pred_Y = sess.run(y,feed_dict={x:x_test})
#         print(pred_Y)
        return pred_Y

### 数据构造

In [ ]:
import itertools
def rise_fall_reg(trade_date_m_series):
    zz_df = dao.get_index_component_data('ZZ')
    stock_price_df = dao.get_security_info('stock_price_info')[trade_date_m_series]
    stock_close_df = stock_price_df.xs('close',level=1)[trade_date_m_series]
    trade_status_df = stock_price_df.xs('trade_status', level=1)[trade_date_m_series]
    pause_df = trade_status_df.copy()
    pause_df[pause_df == 1] = np.nan
    pause_df[pause_df==0] = 1

    stock_chg_df = stock_close_df.pct_change(axis=1)
    stock_return_df = stock_chg_df * pause_df  * zz_df
    stock_return_df = stock_return_df.shift(-1, axis=1)
    return stock_return_df

#划分测试集和训练集
def concat_data(data,date_list,is_rise_df):
    factor_class_series = data.map(lambda x: x.loc[:,date_list[0]:date_list[-1]])
    data_df = pd.DataFrame()
    factor_name_list = factor_class_series.index.tolist()
    data_dict = {}
    for trade_date in tqdm(date_list[:]):
        data_section_series = factor_class_series.map(lambda x: x[trade_date] if trade_date in x.columns else None)
        data_section_df = pd.DataFrame(data_section_series.to_dict())
        data_section_df =data_section_df.reindex(columns=factor_name_list)
        data_section_df['rise_fall'] = is_rise_df[trade_date]
        data_dict[trade_date] = data_section_df
        data_section_df['date'] = trade_date
    return data_dict
    
def data_train_test(data_pct):
    data_pct_test=data_pct      #包含0，-1,1的三种分类的全部数据预测集
    data_pct_test=data_pct[data_pct['trade_status']==0]        #选择正常股票状态的数据
    data_pct_test=data_pct_test.dropna()    #删除空值（回归空值太多）
    data_pct_train=data_pct     #不包含0的训练集
    data_pct_train=data_pct_train[data_pct_train['trade_status']==0]       #选择正常股票状态的数据
    data_pct_train=data_pct_train.dropna()
    return data_pct_train,data_pct_test

def data_input_class(data,date_list):
    is_rise_df=rise_fall_reg(date_list)    
    data_dict=concat_data(data,date_list,is_rise_df)
    data_df = pd.concat([data_dict[frame] for frame in data_dict.keys()])
    train1,test1=data_train_test(data_df) 
    return train1,test1

# 数据标准化，可以处理空值
def standard(X_train,X_test):
    X_train_scaled =1.0 * (X_train - X_train.mean()) / X_train.std()  # 数据标准化
    X_test_scaled =1.0 * (X_test - X_test.mean()) / X_test.std()  # 数据标准化
    return  X_train_scaled,X_test_scaled 

def countsum(data):
    a=data.reset_index()
    a.rename(columns=lambda x:x.replace('index','stock'), inplace=True) 
    resultdata=(a['stock'].groupby(a['date'])).describe()
    resultdata['sum']=resultdata['count'].cumsum()
    return resultdata,a

#两分类划分，划分训练集data，测试集alldata（训练集的类别只有（0,1），测试集包含所有类别（0,1，-1））
def splitdata(data,train,test,i,j,x,y):
    resultdata,a=countsum(train)
    resultalldata,b=countsum(test)
    i=resultdata['sum'][i]
    j=resultdata['sum'][j]
    x=resultalldata['sum'][x]
    y=resultalldata['sum'][y]

    newname=data.index.tolist()    
    newname.append('stock')
    newname.append('date')

    X_train=np.array(a[newname][j:i])
    Y_train=np.array(a['rise_fall'][j:i])
    #第x个月，测试集
    X_test=np.array(b[newname][x:y])
    Y_test=np.array(b['rise_fall'][x:y])
    X_train_scaled,X_test_scaled=X_train[:,:-3],X_test[:,:-3]
    return X_train_scaled,X_test_scaled,Y_train,Y_test,X_test

def change(data,n,m,M,date_list):
    date=date_list[M:-4]
    factor_df=pd.DataFrame(columns=date)
    factor_df['stock']=list(month_alpha_factor_series[100].index)
#     factor_df['stock']=data['stock']

    for i,t in itertools.zip_longest(data,date):
        temp=factor_df[['stock']]
        temp[t]=np.nan
        u=i.iloc[:,[n,m]]
        u.columns=[t,'stock']
        factor_Crash=pd.concat([u,temp],join='inner',ignore_index=True)
        factor_Crash.sort_values(t,inplace=True)
        factor_Crash.drop_duplicates(['stock'],inplace=True)
        factor_Crash.sort_values('stock',inplace=True)
        factor_Crash.reset_index(inplace=True)
        factor_df[t]= factor_Crash[t]
    factorF_df=factor_df.set_index(['stock'])
    return factorF_df

def changeindex1(data,M,date_list):
    factor_df=change(data,0,1,M,date_list)
    return  factor_df

### 因子测试

In [ ]:
import alphafactors.factorprepro_class as fp
import alphafactors.factoranalyse as fa

#因子处理(分成两种方向)
def factor_analyse(name,factor):  # 0-positive , 1-negetive
    factor_prepro_obj = fp.FactorPrePro(factor_name=name, factor_data_df=factor, universe='ZZ', neutral_list=None)
    factor_prepro_obj.run_process(start_date=max(factor.columns[0], dt.datetime(2007,1,5)), end_date=factor.columns[-1])
    df = factor_prepro_obj.factor_pro_df
    factor_test_obj = fa.FactorAnalyse(factor_name=name, factor_data_df=df, factor_dr=0)   # 0-positive , 1-negetive
    factor_test_obj.run_analyse_new(start_date=dt.datetime(2009,1,23), universe='ZZ')
    return factor_test_obj

#因子测试画图显示
def show1(factor_test_obj):
    factor_test_obj.net_value_df.iloc[:,-3:].plot(figsize=(20,10))
def show2(factor_test_obj):
    factor_test_obj.factor_perform_df
    return  factor_test_obj.factor_perform_df
def show3(factor_test_obj):
    factor_test_obj.factor_para_df
    return  factor_test_obj.factor_para_df
def show4(factor_test_obj):
    factor_test_obj.port_perform_df
    return     factor_test_obj.port_perform_df
def show5(factor_test_obj):
    factor_test_obj.port_perform_df['annual_return'].plot(kind='bar')
    return factor_test_obj.port_perform_df['annual_return'].plot(kind='bar')   
def show6(factor_test_obj):
    factor_test_obj.factor_index_df['IC值'].plot(kind='bar', figsize=(20,10), color='blue')
    return  factor_test_obj.factor_index_df['IC值'].plot(kind='bar', figsize=(20,10), color='blue')

#### 因子测试2

In [ ]:
import util.evalstat as evl
                                     
def factor_test_pre(factor):         #因子中性化预处理
    factor_prepro_def stock_choice(data,num):                 #直接挑选概率值前100支股票，等权
    stock_series= pd.Series()
    for i in data.columns:
        stock_series.loc[i]=pd.Series(index=[data[i].sort_values(ascending=False).head(num).index],data=1/num)
    stock_choice_obj=factor_test(stock_series)
    
    return stock_choice_obj

def stock_bench_ind(data,num):   #行业中性，基准权重后挑选100支股票
    stock_series= pd.Series()
    for i in data.columns:
       set_date=i
       stock_series[i]=get_industry_stock(data[i], set_date, stock_num=num)
    stock_choice_obj =factor_test(stock_series)

    return stock_choice_obj

def factor_stock_choose(factor,num):
    factor_obj=factor_test_pre(factor)   #做因子预处理
    #等权选num支
    samew_pre=stock_choice(factor_obj,num)
    samew_unpre=stock_choice(factor,num)   #不做因子预处理，等权直接选100支
    #不等权选num支
    unw_pre=stock_bench_ind(factor_obj,num)
    unw_unpre=stock_bench_ind(factor,num)
    return samew_pre,samew_unpre,unw_pre,unw_unpreobj = fp.FactorPrePro(factor_name='factor_test', factor_data_df=factor, universe='ZZ', neutral_list=None)
    factor_prepro_obj.run_process(start_date=max(factor.columns[0], dt.datetime(2007,1,5)), end_date=factor.columns[-1])
    df = factor_prepro_obj.factor_pro_df
    return df

def factor_test(stock_weighted_series):
    perform_obj=evl.PortPerform(port_series=stock_weighted_series,ret_type='open',fee=0.0035)
    perform_obj.run()
    return perform_obj

def show01(perform_obj):
    perform_obj.net_value_plot()        
def show02(perform_obj):
    perform_obj.get_strategy_perform() 
    return perform_obj.get_strategy_perform()      
def show03(perform_obj):
    perform_obj.get_avg_turnover()  
    return perform_obj.get_avg_turnover()      
def show04(perform_obj):
    perform_obj.get_annual_perform()  
    return  perform_obj.get_annual_perform()  

#### 投资组合构建（2种：一种等权，一种不等权）

In [ ]:
def stock_choice(data,num):                 #直接挑选概率值前100支股票，等权
    stock_series= pd.Series()
    for i in data.columns:
        stock_series.loc[i]=pd.Series(index=[data[i].sort_values(ascending=False).head(num).index],data=1/num)
    stock_choice_obj=factor_test(stock_series)
    
    return stock_choice_obj

def stock_bench_ind(data,num):   #行业中性，基准权重后挑选100支股票
    stock_series= pd.Series()
    for i in data.columns:
       set_date=i
       stock_series[i]=get_industry_stock(data[i], set_date, stock_num=num)
    stock_choice_obj =factor_test(stock_series)

    return stock_choice_obj

def factor_stock_choose(factor,num):
    factor_obj=factor_test_pre(factor)   #做因子预处理
    #等权选num支
    samew_pre=stock_choice(factor_obj,num)
    samew_unpre=stock_choice(factor,num)   #不做因子预处理，等权直接选100支
    #不等权选num支
    unw_pre=stock_bench_ind(factor_obj,num)
    unw_unpre=stock_bench_ind(factor,num)
    return samew_pre,samew_unpre,unw_pre,unw_unpre

#### 基准行业权重

In [ ]:
stock_industry_df = dao.get_security_info('stock_industry_CS')
stock_industry_list = dso.get_industry_classify('CS')
stock_pool_df = dao.get_index_component_data('DEF')

def get_bench_ind_weight(set_date, bench_code='ZZ500'):
    industry_series = optool.get_series_from_df(data_df=stock_industry_df, set_date=set_date, axis=1).dropna()
    group_data = industry_series.index.groupby(industry_series)
    # 基准行业权重
    bench_component_df = dao.get_index_component_data(bench_code)
    bench_series = pd.Series(bench_component_df[set_date].set_index('code')['weight'])
    bench_series = bench_series / bench_series.sum()
    bench_ind_weight_series = pd.Series(index=stock_industry_list)
    for industry_name in stock_industry_list:
        ind_stock_list = group_data[industry_name]
        temp_series = bench_series.copy()
        bench_ind_weight_series.loc[industry_name] = temp_series.reindex(ind_stock_list).sum()
    bench_ind_weight_series.fillna(0, inplace=True)
    return bench_ind_weight_series


def get_industry_stock(stock_factor_series, set_date, stock_num=100):
    industry_series = optool.get_series_from_df(data_df=stock_industry_df, set_date=set_date, axis=1).dropna()
    stock_series = stock_pool_df[set_date]
    stock_list = industry_series.index.intersection(stock_series.dropna().index).tolist()
    industry_series = industry_series.loc[stock_list]

    group_data = industry_series.index.groupby(industry_series)

    # 基准行业权重
    bench_ind_weight_series = get_bench_ind_weight(set_date=set_date)
    bench_ind_num_series = round(bench_ind_weight_series * stock_num).astype(int)

    port_series = pd.Series()
    # 得到行业中性组合
    for industry_name in stock_industry_list[:]:
        if bench_ind_weight_series[industry_name] <= 0.0:
            continue
        ind_stock_list = group_data[industry_name]
        ind_stock_series = pd.Series(stock_factor_series.loc[ind_stock_list]).reindex(ind_stock_list).sort_values(
            ascending=False)

        ind_stock_num = bench_ind_num_series[industry_name]
        if ind_stock_num < 1:
            ind_stock_num += 1

        if ind_stock_series.shape[0] >= ind_stock_num:
            xx = ind_stock_series.head(ind_stock_num)
            temp_series = xx / xx.sum() * bench_ind_weight_series[industry_name]
        else:
            temp_series = ind_stock_series / ind_stock_series.sum() * bench_ind_weight_series[industry_name]

            temp_series = pd.Series(index=ind_stock_series.index[:ind_stock_num], data=1.0 / ind_stock_num) * \
                          bench_ind_weight_series[industry_name]
        port_series = port_series.append(temp_series)
    return port_series